## Importing

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

In [7]:
df = pd.read_csv('C:\Users\motaz\OneDrive\Desktop\Projects\Sales_prediction\Train')
df

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 3-4: truncated \UXXXXXXXX escape (1367367133.py, line 1)